^fill^README.md^here^

- - -

^fill^theory.md^here^

- - -

## Demonstration

### 1. Library imports

Import libraries needed by the calculation. The external libraries used are:

- [numpy](http://www.numpy.org/)

- [DataModelDict](https://github.com/usnistgov/DataModelDict)

- [atomman](https://github.com/usnistgov/atomman)

- [iprPy](https://github.com/usnistgov/iprPy)

In [1]:
# Standard library imports
from __future__ import division, absolute_import, print_function
import os
import sys
import uuid
import glob
import shutil
import datetime
from copy import deepcopy

# http://www.numpy.org/
import numpy as np 

# https://github.com/usnistgov/DataModelDict 
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

### 2. Assign values for the calculation's run parameters

#### 2.0 Set the calculation's working directory

In [2]:
calc_name = 'point_defect_structure'

# Check current working directory
cwd_name = os.path.basename(os.getcwd())

# Change working directory if needed
if cwd_name != calc_name:
    if not os.path.isdir(calc_name):
        os.mkdir(calc_name)
    os.chdir(calc_name)

#### 2.1 Specify system-specific paths

- __lammps_command__ (required) is the LAMMPS command to use.

- __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

- __lib_directory__ defines the relative path to the iprPy library. This makes it easier to define paths to reference records later.

In [3]:
lammps_command = 'lmp_serial'
mpi_command = None
lib_directory = '../../../library'

#### 2.2 Specify the prototype system

- __system__ (required) is an atomman.System to perform the scan on. 

- __ucell__ (required) is an atomman.System representing a fundamental unit cell of the system.

- __prototype_name__ gives the name of the crystal_prototype reference record in the iprPy library to use for ucell. 

- __prototype_path__ gives the path to the crystal_prototype reference record to use.

- __box_parameters__ defines the initial guess box parameters to scale ucell and system by.

- __sizemults__ list of three integers specifying how many times the ucell vectors of $a$, $b$ and $c$ are replicated in creating system.

In [4]:
prototype_name = 'A1--Cu--fcc'
box_parameters = [3.52, 3.52, 3.52]
sizemults = [5,5,5]

# Define prototype_path using lib_directory and prototype_name
prototype_path = os.path.abspath(os.path.join(lib_directory, 'crystal_prototype', prototype_name+'.json'))

# Create ucell by loading prototype record
ucell = am.load('system_model', prototype_path)[0]
print('# of atoms in ucell =', ucell.natoms)

# Rescale ucell using box_parameters
ucell.box_set(a=box_parameters[0], b=box_parameters[1], c=box_parameters[2], scale=True)

# Generate system by supersizing ucell
system = am.supersize(ucell, *sizemults)
print('# of atoms in system =', system.natoms)

# of atoms in ucell = 4
# of atoms in system = 500


#### 2.3 Specify the potenital and elemental symbols

- __potential__ (required) is the atomman.lammps.Potential representation of a LAMMPS implemented potential to use.

- __symbols__ (required) is a list of the elemental model symbols of potential to associate with the unique atom types of system.

- __potential_name__ gives the name of the potential_LAMMPS reference record in the iprPy library to use for potential. 

- __potential_path__ gives the path to the potential_LAMMPS reference record to use.

- __potential_dir_path__ gives the path for the folder containing the artifacts associated with the potential (i.e. eam.alloy file)



In [5]:
potential_name = '1999--Mishin-Y--Ni--LAMMPS--ipr1'
symbols = ['Ni']

# Define potential_path and potential_dir_path using lib_directory and potential_name
potential_dir_path = os.path.abspath(os.path.join(lib_directory, 'potential_LAMMPS', potential_name))
potential_path = potential_dir_path + '.json'

# Create potential by loading LAMMPS-potential record
with open(potential_path) as f:
    potential = lmp.Potential(f, potential_dir_path)
print('Successfully loaded potential', potential)

Successfully loaded potential 1999--Mishin-Y--Ni--LAMMPS--ipr1


#### 2.4 Specify the defect 

__2.4a Specify the defect model__

- __pointdefect_model__ is a DataModelDict of a point-defect record.

- __pointdefect_name__ gives the name of the point-defect reference record in the iprPy library to use for potential. 

- __pointdefect_path__ gives the path to the point-defect reference record to use.

In [6]:
pointdefect_name = 'A1--Cu--fcc--vacancy'
#pointdefect_name = 'A1--Cu--fcc--100-dumbbell'
#pointdefect_name = 'A1--Cu--fcc--110-dumbbell'
#pointdefect_name = 'A1--Cu--fcc--111-dumbbell'
#pointdefect_name = 'A1--Cu--fcc--octahedral-interstitial'
#pointdefect_name = 'A1--Cu--fcc--tetrahedral-interstitial'
#pointdefect_name = 'A1--Cu--fcc--crowdion-interstitial'

# Define pointdefect_path using lib_directory and pointdefect_name
pointdefect_path = os.path.abspath(os.path.join(lib_directory, 'point_defect', pointdefect_name+'.json')) 

# Load pointdefect record as a DataModelDict
with open(pointdefect_path) as f:
    pointdefect_model = DM(f)

print('Successfully loaded defect parameters for', pointdefect_model['point-defect']['id'])

# Extract defect parameters    
point_kwargs = pointdefect_model['point-defect']['calculation-parameter']
print(point_kwargs.json(indent=4))

Successfully loaded defect parameters for A1--Cu--fcc--vacancy
{
    "ptd_type": "v", 
    "pos": "0.0000000000000 0.0000000000000 0.0000000000000", 
    "scale": true
}


__2.4b Specify the point defect parameters__

- __point_kwargs__ (required) is a dictionary or list of dictonaries containing parameters for generating the defect. Values can either be extracted from a point-defect model (as done here), or defined independently. Allowed keywords are:

    - __ptd_type__ indicates which defect type to generate: 'v' for vacancy, 'i' for interstitial, 's' for substitutional, or 'db' for dumbbell.
    
    - __atype__ is the atom type to assign to the defect atom ('i', 's', 'db' ptd_types).
    
    - __pos__ specifies the position for adding the defect atom (all ptd_types).
    
    - __ptd_id__ specifies the id of an atom in the initial system where the defect is to be added. Alternative to using pos ('v', 's', 'db' ptd_types).
    
    - __db_vect__ gives the vector associated with the dumbbell interstitial to generate ('db' ptd_type).
    
    - __scale__ indicates if pos and db_vect are in absolute (False) or box-relative (True) coordinates. Default is False.
    
    - __atol__ is the absolute tolerance for position-based searching. Default is 1e-3 angstroms.

In [7]:
if not isinstance(point_kwargs, (list, tuple)):
    point_kwargs = [point_kwargs]

#Unscale vector parameters relative to ucell
for i in xrange(len(point_kwargs)):
    params = point_kwargs[i]
    
    scale = iprPy.input.boolean(params.get('scale', False))
    
    if 'atype' in params:
        params['atype'] = int(params['atype'])

    if 'pos' in params:
        params['pos'] = np.array(params['pos'].strip().split(), dtype=float)
        if scale is True:
            params['pos'] = ucell.unscale(params['pos'])
    if 'db_vect' in params:

        params['db_vect'] = np.array(params['db_vect'].strip().split(), dtype=float)
        if scale is True:
            params['db_vect'] = ucell.unscale(params['db_vect'])

    params['scale'] = False
        
for params in point_kwargs:
    print('{')
    for key, value in params.iteritems():
        print('   ',key,':',value)
    print('}')

{
    ptd_type : v
    pos : [ 0.  0.  0.]
    scale : False
}


#### 2.5 Specify calculation-specific run parameters

- __energytolerance__ is the energy tolerance to use during the minimizations. This is unitless.

- __forcetolerance__ is the force tolerance to use during the minimizations. This is in energy/length units.

- __maxiterations__ is the maximum number of minimization iterations to use.

- __maxevaluations__ is the maximum number of minimization evaluations to use.

- __maxatommotion__ is the largest distance that an atom is allowed to move during a minimization iteration. This is in length units.

In [8]:
energytolerance = 1e-8
forcetolerance = uc.set_in_units(0.0, 'eV/angstrom')
maxiterations = 10000
maxevaluations = 100000
maxatommotion = uc.set_in_units(0.01, 'angstrom')

### 3. Define calculation function(s) and generate template LAMMPS script(s)

#### 3.1 min.template

In [9]:
with open('min.template', 'w') as f:
    f.write("""^fill^min.template^here^""")

#### 3.2 pointdefect()

In [10]:
^fill^calc_point_defect_static.py(pointdefect)^here^

#### 3.3 check_ptd_config()

In [11]:
^fill^calc_point_defect_static.py(check_ptd_config)^here^

### 4. Run calculation function(s)

#### 4.1 Generate point defect system and evaluate the energy

In [12]:
results_dict = pointdefect(lammps_command, 
                           system, 
                           potential,
                           symbols,
                           point_kwargs,
                           mpi_command = mpi_command,
                           etol = energytolerance,
                           ftol = forcetolerance,
                           maxiter = maxiterations,
                           maxeval = maxevaluations,
                           dmax = maxatommotion)

In [13]:
results_dict.keys()

['E_total_ptd',
 'E_coh',
 'system_ptd',
 'dumpfile_ptd',
 'system_base',
 'E_ptd_f',
 'E_total_base',
 'dumpfile_base']

#### 4.2 Characterize if the defect has reconfigured

In [14]:
cutoff = 1.05*ucell.box.a
results_dict.update(check_ptd_config(results_dict['system_ptd'], 
                                     point_kwargs, 
                                     cutoff))

In [15]:
results_dict.keys()

['has_reconfigured',
 'E_total_ptd',
 'E_coh',
 'system_ptd',
 'dumpfile_ptd',
 'system_base',
 'centrosummation',
 'E_ptd_f',
 'E_total_base',
 'dumpfile_base']

### 5. Report results

#### 5.1 Define units for outputting values

- __length_unit__ is the unit of length to display relaxed lattice constants in.
- __energy_unit__ is the unit of energy to display cohesive energies in.

In [16]:
length_unit = 'angstrom'
energy_unit = 'eV'

#### 5.2 Print $E_{coh}$ and $E_{ptd}^f$

In [17]:
print('Ecoh =  ', uc.get_in_units(results_dict['E_coh'], energy_unit), energy_unit)
print('Eptd_f =', uc.get_in_units(results_dict['E_ptd_f'], energy_unit), energy_unit)

Ecoh =   -4.44999999835 eV
Eptd_f = 1.60000249825 eV


#### 5.3 Check configuration parameters

In [18]:
print('Has the system (likely) reconfigured?', results_dict['has_reconfigured'])
if 'centrosummation' in results_dict:
    print('centrosummation =', uc.get_in_units(results_dict['centrosummation'], length_unit), length_unit)
if 'position_shift' in results_dict:
    print('position_shift = ', uc.get_in_units(results_dict['position_shift'], length_unit), length_unit)
if 'db_vect_shift' in results_dict:
    print('db_vect_shift =  ', uc.get_in_units(results_dict['db_vect_shift'], length_unit), length_unit)

Has the system (likely) reconfigured? False
centrosummation = [  1.67976425e-14   1.86517468e-14   1.95399252e-14] angstrom
